In [2]:
import numpy as np
import pandas as pd
import random
from IPython import display
from collections import namedtuple, deque
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
device = torch.device("cpu")
# use cpu run
import gym

In [3]:
env_args = {
    'run_dssat_location': '/opt/dssat_pdi/run_dssat',  # assuming (modified) DSSAT has been installed in /opt/dssat_pdi
    'log_saving_path': './logs/dssat-pdi.log',  # if you want to save DSSAT outputs for inspection
    # 'mode': 'irrigation',  # you can choose one of those 3 modes
    # 'mode': 'fertilization',
    'mode': 'all',
    'seed': 123456,
    'random_weather': False,  # if you want stochastic weather
    #'fileX_template_path':'./IUAF9901.MZX',
    'fileX_template_path':'./SIAZ9501.MZX',
    #'experiment_number':1
}
env = gym.make('gym_dssat_pdi:GymDssatPdi-v0', **env_args)
print('Observation:',env.observation,)
print(len(env.observation),len(env.observation['sw']))
ram_dimensions = len(env.observation)+len(env.observation['sw'])-1
#ram_dimnetion = 9 if partial
nb_actions = 25
print('\nRam information received from DASSAT will has %d dimensions.' % ram_dimensions)
print('There are %d possible actions at each step.' % nb_actions)
print('Discrete?',type(gym.spaces)== gym.spaces.Discrete)
# observation has 27 elements, 9 values in soil water
# state size = 27+8 dimension
# how to defind nb_action? why is 200?

Observation: {'cleach': 0.0, 'cnox': 0.0, 'cumsumfert': 0.0, 'dap': 0, 'dtt': 0.0, 'es': 0.0, 'grnwt': 0.0, 'istage': 7, 'nstres': 0.0, 'pcngrn': 0.0, 'pltpop': 8.0, 'rain': 1.2000000476837158, 'rtdep': 0.0, 'runoff': 0.0, 'srad': 19.799999237060547, 'sw': array([0.20299999, 0.20299999, 0.20299999, 0.28200001, 0.28200001,
       0.223     , 0.223     ]), 'swfac': 0.0, 'tleachd': 0.0, 'tmax': 24.600000381469727, 'tmin': 12.199999809265137, 'tnoxd': 0.0, 'topwt': 0.0, 'trnu': 0.0, 'vstage': 0.0, 'wtdep': 0.0, 'wtnup': 0.0, 'xlai': 0.0}
27 7

Ram information received from DASSAT will has 33 dimensions.
There are 25 possible actions at each step.
Discrete? False


In [11]:
for i in range(1):
    #print('')
    rain=0
    date=1
    n_date=0
    env.reset()
    c1=1;
    c2=0
    rain=rain+env.observation['rain']
    #print(env.observation)
    #date = 1
    n_list=[]
    w_list=[]
    istage_list=[]
    vstage_list=[]
    w_amount=0
    while not env.done:
        #print(observation)
        #print(observation)
        # observation_list = env.observation_dict_to_array(observation)
        w_input=0
        n_input=0
        if c1!=0:
            n_input=50
            c1=c1-1
            #print('n 50')
        if date==166-136:
            n_input = 100
            #print('n 100')
        if date == 191-136:
            n_input = 100
            #print('n 100')
        if date%3==0:
            w_input=16
            #print(w_input)
            c2=c2+1
        action = {
                    'anfer': n_input,  # if mode == fertilization or mode == all ; nitrogen to fertilize in kg/ha4
                    'amir': w_input,  # if mode == irrigation or mode == all ; water to irrigate in L/ha
            }
        #print(date)
        observation, reward, done, info = env.step(action)
        date = date+1
        #print(observation['yrdoy'])
        #print('')
        #print(observation)
        #real_ob = add_fert(observation, action_dict['anfer'])
        #print('real_ob is ', real_ob)
        #env.observation = real_ob
        #print('final ob is:', env.observation)
        #real_s=get_real_state(env._state, action_dict['anfer'])
        #env._state=real_s
        #print('env_state', env._state)
        #print(date)
        #print(reward)
        #print(done)
        if done:
            final_ob=env.observation
            print(final_ob)
            N_input=final_ob['cumsumfert']
            Nleach=final_ob['cleach']
            Yield=final_ob['grnwt']
            reward=Yield*0.158-0.79*N_input-1.1*w_amount
            print('irrigation time is ', c2)
            #print(env.history['action'])
            #print(env.history['reward'])
            break
        w_amount = w_amount+w_input
        n_list.append(action['anfer'])
        w_list.append(action['amir'])
        istage_list.append(env.observation['istage'])
        vstage_list.append(env.observation['vstage'])
        rain=rain+env.observation['rain']
#print(np.mean(rain_list))
print('N amount is', N_input)
print('N action is', n_list)
print('W amount is', w_amount)
print('W action is', w_list)
#print('avergae N input is', np.mean(Action_list))
#print('')
print('N leaching is', Nleach)
#print('avergae Leaching is', np.mean(Nleach_list))
#print('')
print('Yield is', Yield)
#print('avergae Yield is', np.mean(Yield_list))
#print('')
print('Reward is', reward)
#print('avergae reward is', np.mean(reward_list))
# import pandas
# df = pandas.DataFrame(n_list, columns=["colummn"])
# df.to_csv('n_list.csv', index=False)
# df = pandas.DataFrame(w_list, columns=["colummn"])
# df.to_csv('w_list.csv', index=False)
# df = pandas.DataFrame(istage_list, columns=["colummn"])
# df.to_csv('istage_list.csv', index=False)
# df = pandas.DataFrame(vstage_list, columns=["colummn"])
# df.to_csv('vstage_list.csv', index=False)

{'cleach': 4.092536926269531, 'cnox': 0.012328286655247211, 'cumsumfert': 250.00003051757812, 'dap': 140, 'dtt': 11.75, 'es': 0.49474215507507324, 'grnwt': 10989.937744140625, 'istage': 6, 'nstres': 0.3285132050514221, 'pcngrn': 0.015422229766845702, 'pltpop': 8.0, 'rain': 0.0, 'rtdep': 80.0011978149414, 'runoff': 0.0, 'srad': 13.399999618530273, 'sw': array([0.27751601, 0.277482  , 0.27211601, 0.27595201, 0.27311799,
       0.26772499, 0.26773199]), 'swfac': 0.0, 'tleachd': 0.13787662982940674, 'tmax': 27.5, 'tmin': 12.0, 'tnoxd': 0.0, 'topwt': 2655.3046875, 'trnu': 0.22487860172986984, 'vstage': 22.0, 'wtdep': 80.0011978149414, 'wtnup': 319.49575424194336, 'xlai': 2.604471445083618}
irrigation time is  47
N amount is 250.00003051757812
N action is [50, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [4]:
env.reset()
print(env.observation)

{'cleach': 0.0, 'cnox': 0.0, 'cumsumfert': 0.0, 'dap': 0, 'dtt': 0.0, 'es': 0.0, 'grnwt': 0.0, 'istage': 7, 'nstres': 0.0, 'pcngrn': 0.0, 'pltpop': 7.199999809265137, 'rain': 0.0, 'rtdep': 0.0, 'runoff': 0.0, 'srad': 13.300000190734863, 'sw': array([0.086     , 0.086     , 0.086     , 0.086     , 0.086     ,
       0.076     , 0.076     , 0.13      , 0.25799999]), 'swfac': 0.0, 'tleachd': 0.0, 'tmax': 22.200000762939453, 'tmin': 3.299999952316284, 'tnoxd': 0.0, 'topwt': 0.0, 'trnu': 0.0, 'vstage': 0.0, 'wtdep': 0.0, 'wtnup': 0.0, 'xlai': 0.0}


[212.58120727539062]

[10771.492390434141]

[984.4257976885943]

'1happy.csv'

-100